<a href="https://colab.research.google.com/github/Pelosomelo/Misp/blob/main/plan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem podziału godzin
W firmie jest kilku pracowników. Każdy z nich ma określoną ilość godzin każdego dnia jaką może maksymalnie spędzić w pracy. Każdy z pracowników musi pracować 8 godzin tygodniowo. Ponadto biuro otwarte jest jedynie przez 14 godzin przez 5 dni w tygodniu i znajduje się w nim tylko jeden komputer, więc suma pracowanych godzin wszystkich pracowników nie może przekraczać 14 godzin dziennie.

Tak sformułowane zadanie nie znajdzie jednak rozwiązania dla więcej niż 8 pracowników ze względu na ilość godzin otwarcia biura w tygodniu.

## Wersja rozszerzona o obsługę danych z pliku csv zawierającego dowolną ilością pracowników

### Import bibliotek używanych do rozwiązania problemu

In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

In [ ]:
from pulp import *
import pandas as pd
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import danych o godzinach z csv, sortowanie pracowników alfabetycznie oraz obliczenie ilości pracowników

In [ ]:
dzien = pd.read_csv('/content/drive/MyDrive/tablica_z_dodanym.csv',index_col=0).to_dict()
pon = dzien['1_PN']
Pracownicy=list(pon.keys())
Pracownicy.sort()
ilosc=len(Pracownicy)
print(dzien)

{'1_PN': {'KC': 6, 'DH': 0, 'HB': 4, 'SC': 5, 'KS': 3, 'NK': 0, 'AA': 2}, '2_WT': {'KC': 0, 'DH': 6, 'HB': 8, 'SC': 5, 'KS': 0, 'NK': 0, 'AA': 2}, '3_SR': {'KC': 6, 'DH': 0, 'HB': 4, 'SC': 5, 'KS': 3, 'NK': 0, 'AA': 2}, '4_CW': {'KC': 0, 'DH': 6, 'HB': 0, 'SC': 0, 'KS': 8, 'NK': 6, 'AA': 2}, '5_PT': {'KC': 6, 'DH': 0, 'HB': 4, 'SC': 5, 'KS': 0, 'NK': 2, 'AA': 2}}


### Deklaracja zmiennych

In [ ]:
#Pracownicy = ["KC", "DH", "HB", "SC", "KS", "NK"]
Dni = ["1_PN", "2_WT", "3_SR", "4_CW", "5_PT"]
czas = [[0 for col in range(5)] for row in range(ilosc)]
wynik = [[0 for col in range(5)] for row in range(ilosc)]
for x in range(ilosc):
    for y in range(5):
        czas[x][y]=dzien[Dni[y]][Pracownicy[x]]
max={'1_PN':9,'2_WT':10,'3_SR':11,'4_CW':12,'5_PT':13}

### Sformułowanie problemu liniowego

In [ ]:
czas = makeDict([Pracownicy,Dni],czas,0)
prob = LpProblem("Grafik",LpMinimize)
vars = LpVariable.dicts("Grafik",(Pracownicy,Dni),0,None,LpInteger)

### Wprowadzenie ograniczeń

In [ ]:
for w in Pracownicy:
    prob += lpSum([vars[w][b] for b in Dni])==8

for b in Dni:
    prob += lpSum([vars[w][b] for w in Pracownicy])==max[b]

for w in Pracownicy:
    for b in Dni:
        prob += vars[w][b]<=czas[w][b]

### Rozwiązanie problemu
Specyfikacje zdefiniowanego problemu zapiszemy w pliku pod nazwą "Grafik_Pracownikow", rozwiązujemy problem i na ekranie wyświetlamy informację o statusie rozwiązania. Jeżeli wyświetli nam się "Status: Optimal" oznacza to, że wynik jest poprawny. Gdyby pojawił się "Status: Infeasible" oznacza to że wynik jest niemożliwy do otrzymania i wtedy potrzebna jest redefinicja problemu z innymi warunkami.

In [ ]:
prob.writeLP("Grafik_Pracownikow.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Infeasible


### Przygotowanie wyniku
Wynik zapisany jest w (5 * ilość pracowników) zmiennych, przepisując wartości zmiennych do tablicy, możemy je posortować i wyświetlić w wybranej przez nas kolejności.

In [ ]:

    # Każda zmienna zostanie zapisana do tablicy wynik
for v in range(ilosc*5):
    wynik[math.floor(v/5)][v%5] = prob.variables()[v].varValue
    


### Wyświetlenie wyniku
Za pomocą jednej pętli i wyświetlonemu najpierw nagłówkowi, możemy przedstawić wynik w formie czytelnej tabeli.

In [ ]:
print("    PN   WT   SR   CZW  PT")
for i in range(ilosc):
    print(Pracownicy[i],wynik[i])
        

    PN   WT   SR   CZW  PT
AA [2.0, 2.0, 2.0, 0.0, 1.0]
DH [0.0, 4.0, 0.0, 4.0, 0.0]
HB [0.0, 0.0, 4.0, 0.0, 4.0]
KC [0.0, 0.0, 2.0, 0.0, 6.0]
KS [3.0, 0.0, 3.0, 2.0, 0.0]
NK [0.0, 0.0, 0.0, 6.0, 2.0]
SC [4.0, 4.0, 0.0, 0.0, 0.0]
